# CHEM 1000 - Fall 2020
Prof. Geoffrey Hutchison, University of Pittsburgh

## 1. Introduction

### Unit Conversion

Unit conversion is usually a major annoyance in physical sciences. Fortunately, it's possible in Python to track and convert units using a package called `pint` (hah.)

`pint` tracks a [lot of units](https://github.com/hgrecco/pint/blob/master/pint/default_en.txt) and most standard prefixes (e.g., *pico-* to *giga-* and more).

Here's a quick tutorial - there's more [documentation online](https://pint.readthedocs.io/en/stable/index.html) including a more complete [tutorial](https://pint.readthedocs.io/en/stable/tutorial.html).

In [1]:
###
# This is a bit of code to make sure the most recent version of pint is installed
#  if not, install it
# You can safely ignore the code...

install = False
try:
    import pint 
    def versiontuple(v):
        return tuple(map(int, (v.split("."))))

    if versiontuple(pint.__version__) < versiontuple('0.14'):
        install = True
except ModuleNotFoundError:
    install = True
    
if install:
    import sys
    !{sys.executable} -m pip install pint --upgrade

In [2]:
import pint

ureg = pint.UnitRegistry()

In [22]:
# we can convert this
car_speed = 55*ureg.miles/ureg.hour
print(car_speed.to('km/hr'))
print(car_speed.to_base_units())

# alternately
Q_ = ureg.Quantity
Q_(55, 'miles/hour').to_base_units()

88.51 kilometer / hour
24.59 meter / second


24.587200000000003 <Unit('meter / second')>

I'm considering importing a car - it claims 5 L/100km fuel efficiency, but I know miles per gallon...

In [4]:
ureg.define('mpg = 1 * mile / gallon')
ureg.define('_100km = 100 * kilometer')
fuel_ec_europe = 5 * ureg.L / ureg._100km

(1 / fuel_ec_europe).to(ureg.mpg)

47.042916666666656 <Unit('mpg')>

In [5]:
light = Q_("299792458 m/s")
print(light.to("miles/hour"))

670616629.384395 mile / hour


Okay, I think that breaks the speed limit around here...

In [6]:
print(light.to("miles/second"))

186282.39705122085 mile / second


In [7]:
# ooh, let's figure out the lightning flash thing..
# essentially, light travels so fast, that it's effectively instantaneous
time = 1.0*ureg.mile / light
print('light 1 mile in: ', time.to_base_units().to_compact())

# so let's figure out how far the sound travels in a second
# Wikipedia gives ~346 m/s on a summer day: https://en.wikipedia.org/wiki/Speed_of_sound#Tables
sound = Q_(346, 'm/s')
print(sound.to('miles/s'))

light 1 mile in:  5.368193752225749 microsecond
0.21499443251411754 mile / second


So, if you count ~5 seconds between the flash and the thunder, the lighting is 1 mile away

In [8]:
# some math, with conversion between mixed units
floor_area = 12*ureg.feet * 10*ureg.meter
floor_area.to_base_units()

36.57599999999999 <Unit('meter ** 2')>

In [9]:
# what about density?
# aka, do I float?

weight = 165*ureg.lbs
height = 6*ureg.ft + 2*ureg.inches
width = 18*ureg.inches # honestly, I have no idea
depth = 5*ureg.inches # a random guess

volume = height * width * depth # well, no, but let's assume I'm a box
density = weight / volume
print(density.to_base_units())
print(density.to('g/mL'))

685.76340498251 kilogram / meter ** 3
0.6857634049825102 gram / milliliter


In [10]:
# temperature conversion
weather = Q_(82, "degF") # degrees Fahrenheit
ureg.default_format = '.2f' # number of decimals
print(weather.to('degC'))
print(weather.to('kelvin'))

27.78 degree_Celsius
300.93 kelvin


Batteries are often listed with somewhat weird capacity units - milli-Ampere•hour (e.g., they can supply an electrical current of 1 mA for X number of hours. Here's my iPhone:

In [11]:
capacity = Q_('3110 mA*hr')
print(capacity)

3110.00 hour * milliampere


The weird thing is that electrical current is charge $\times$ time, so we should be able to convert that to Coulombs (i.e., units of charge):

In [12]:
capacity.to('coulomb')

11195.999999999998 <Unit('coulomb')>

Let's create a new unit - so we can figure out how many electrons are in my iPhone battery...

In [13]:
ureg.define('charge = 1.0/6.2415090744e18 coulomb')
print(capacity.to('charges'))

print('moles ', capacity.to('charges').magnitude / 6.023e23)

69879935596982385508352.00 charge
moles  0.11602180906023973


A little over 0.1 mole, which seems fairly reasonable. We could, if we wanted, work out charge density (volume) or charg density per gram, etc.

Here's a personal favorite. The atomic unit of electrical field corresponds to the field between a proton and an electron in a hydrogen atom.

In [14]:
au = Q_('1.0 atomic_unit_of_electric_field')
# the "to_compact" method will add prefixes as needed
au.to_compact('V/m')

514.220674762318 <Unit('gigavolt / meter')>

In [15]:
au.to('V/nm')

514.2206747623179 <Unit('volt / nanometer')>

That's obviously pretty big. What if we have one charge (e.g. a Na+ ion) at one nanometer from a molecule:

In [16]:
ion_field = Q_(1.0, "V/nm")
ion_field.to('atomic_unit_of_electric_field')

0.0019446903811524442 <Unit('atomic_unit_of_electric_field')>

So ~0.002 times the proton-electron field are clearly pretty normal

### Energy conversions:

1 Hartree is the "atomic unit" of energy

In [17]:
energy = Q_(1.0, 'hartree')
energy.to('eV')

27.21138624598847 <Unit('electron_volt')>

In [18]:
energy.to('kcal') * 6.023e23/ureg.mol

627.5990072144787 <Unit('kilocalorie / mole')>

### Spectroscopy conversions:


In [19]:
q = 500 * ureg.nm
# q.to('Hz') # this will give a 'DimensionalityError' because nm is a length and Hz is a frequency
q.to('Hz', 'spectroscopy')

599584915999999.9 <Unit('hertz')>

In [20]:
# you don't really need the spectroscopy part
wavelength = 550 * ureg.nm
frequency = (ureg.speed_of_light / wavelength).to('Hz')
print(frequency)
print(frequency.to_compact())

545077196363636.25 hertz
545.08 terahertz


In [21]:
# we see a peak in our IR spectra at 1600 wavenumbers
ir = 1600 * ureg.cm_1
ir.to('nm', 'spectroscopy')

6249.999999999999 <Unit('nanometer')>

-------
This notebook is from Prof. Geoffrey Hutchison, University of Pittsburgh
https://github.com/ghutchis/chem1000

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.